In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os

In [2]:
path = 'data/'
title = []
date = []
report = []

In [3]:
for filename in os.listdir(path):
    if filename != '.DS_Store':
        soup = BeautifulSoup(open(path + filename,encoding = "ISO-8859-1"),"lxml")
        # get header (title + date)
        letters = soup.find_all(class_="header")[0].string
        letters_split = letters.split(' ')
        date_ = letters_split[-1]
        title_ = ' '.join([i for i in letters_split[:-1] if i != ''])
        # get report
        # note: I did not remove '\n' in case we will use it in the future (e.g. paragraphs).
        tables = soup.find_all("table", {"cellpadding": "10"})
        text = tables[1].findAll('td')[0].text
        text = text.replace('\xa0', '')
        title.append(title_)
        date.append(date_)
        report.append(text)

In [4]:
result = pd.DataFrame({'title': title, 'date':date, 'report':report})

In [5]:
result.head()

,date,report,title
0,10-10-16,\n\n\t\t\n\t\tHi Everyone ~\n\nWarm temperatur...,"Junctions, Prospects, Blacktail Prospects"
1,10-11-16,\n\n\t\t\n\t\tHi Everyone ~\n\nBig change in t...,"Prospect Peak, ? Wolves in Hayden, grizzlies"
2,10-12-16,\n\n\t\t\n\t\tHi Everyone ~\n\nClear sunny ski...,"Canyons, Prospect, Junctions, Wapiti, ?wolves"
3,10-13-16,"\n\n\t\t\n\t\tHi Everyone ~\n\nWell, you spend...","Prospect 996M Group, Junctions, Prospect Black..."
4,10-14-16,"\n\n\t\t\n\t\tHi Everyone ~\n\nWell, it truly ...","Lamar Canyon, grizzly"


In [6]:
#The one that DOES NOT work

import nltk.data, nltk.tag

from nltk.tag.perceptron import PerceptronTagger

from nltk.corpus import brown

patterns = [
    
#     (r'.*ing$','VBG'),
#     (r'.*ed$','VBD'),
#     (r'.*es$','VBZ'),
#     (r'.*ould$','MD'),
#     (r'.*\'s$','NN$'),
#     (r'.*s$','NNS'),
#     (r'.*','NN')
    
    
    
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
         (r'(The|the|A|a|An|an)$', 'AT'),   # articles
         (r'.*able$', 'JJ'),                # adjectives
         (r'.*ness$', 'NN'),                # nouns formed from adjectives  
         (r'.*ly$', 'RB'),                  # adverbs
         (r'.*s$', 'NNS'),                  # plural nouns  
         (r'.*ing$', 'VBG'),                # gerunds   
         (r'.*ed$', 'VBD'),                 # past tense verbs
         (r'.*', 'NN')                      # nouns (default)
        
    
]

/Users/RK/anaconda/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [28]:
import csv

mydict = {}
reader = csv.reader(open("POS_tagging.csv", "r"))
for rows in reader:
    k = rows[0]
    v = rows[1]
    mydict[k] = v
print(mydict)

{'Geyser Basin': 'PLC', '996M': 'WLF', 'Prospect': 'WLFP', 'bear': 'BEAR', 'Hitching Post': 'PLC', 'Lamar': 'PLC', 'pup': 'WLF', 'Junction Buttes': 'WLFP', '925M': 'WLF', 'Mary Mountain Trail': 'PLC', 'Dead Puppy Hill': 'PLC', 'magpies': 'BIRD', 'Wapiti': 'ELK', 'Nine Mile': 'PLC', 'Divide Ridge': 'PLC', '949M': 'WLF', 'Cow Elk': 'Elk', '966M': 'WLF', '926F': 'WLF', 'ravens': 'BIRD', 'Buffalo Ford': 'PLC', 'coyotes': 'COY', '821F': 'WLF', 'Cache Valley': 'PLC', 'Canyons': 'WLFP', '763M': 'WLF', 'Mollies Pack': 'WLFP', '964M': 'WLF', 'grizzly': 'BEAR', 'Skyline': 'PLC', '969F': 'WLF', 'Antelope': 'PLC', 'Prospects': 'WLFP', 'Cub Creek': 'PLC', 'Coyote': 'PLC', 'Hubbard Hill': 'PLC', 'Junctions': 'WLFP', 'Hayden Valley': 'PLC', 'Round Prairie': 'PLC', 'Fountain Flats': 'PLC', 'South Entrance': 'PLC', 'Specimen Ridge': 'PLC', 'Long Pullout': 'PLC', "Fishermen's": 'PLC', 'Junction Lake': 'PLC', 'Cascade Meadow': 'PLC', 'Jasper Bench': 'PLC', 'moose': 'MOOSE', 'LittleT': 'WLF', '712M': 'WLF

In [8]:
fd = nltk.FreqDist(brown.words())
cfd = nltk.ConditionalFreqDist(brown.tagged_words())
most_freq_words = fd.most_common(10000)
likely_tags = dict((word, cfd[word].max()) for (word, _) in most_freq_words)
#default_tagger = nltk.UnigramTagger(model=likely_tags)
#default_tagger = nltk.DefaultTagger('NN')
default_tagger = nltk.RegexpTagger(patterns)

In [9]:
def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))          
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))
    return entity_names

In [51]:




for entry in result['report']:
    
    paragraph = entry.split("\n")
    paragraph = [x for x in paragraph if x != '']
    #print(paragraph[:10])
    
    for index, i in enumerate(paragraph):
        print('paragraph #' + str(index + 1))
        #print(i)
        #print('testing')
        entity_names = []

        #print(entry,'...\n\n---------------------\n')
        tagged_sentences = []
        _POS_TAGGER = 'taggers/maxent_treebank_pos_tagger/english.pickle'
        sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
        sent_list = sent_detector.tokenize(i.strip())   

        tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sent_list]

        model = mydict

        tagger = nltk.tag.UnigramTagger(model=model, backoff=default_tagger)
        tagged_sentences = [tagger.tag(sentence) for sentence in tokenized_sentences]
        chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)

        things = []
        places = []

        for sentence in tagged_sentences:

            for item in sentence:

                if item[1] == 'WLFP' or item[1] == 'WLF'or item[1] == 'MOOSE'or item[1] == 'BEAR':
                    things.append(item) 

                if item[1] == 'PLC':
                    places.append(item[0]) 

        if len(things) > 0 or len(places) > 0:
            print(i)
            print("Things in report:", set(things))
            print("Places in report:", set(places),"\n")


        #for word in chunked_sentences:
        #    entity_names.extend(extract_entity_names(word)) 
        #print("potential other objects and places",entity_names)
    
    break
   


paragraph #1
paragraph #2
paragraph #3
paragraph #4
Nothing in Lamar this morning with only very faint signals on 926F and 949M.
Things in report: {('949M', 'WLF'), ('926F', 'WLF')}
Places in report: {'Lamar'} 

paragraph #5
Junctions ~ The story was different over at Slough where there was a great chance that we would find the Junction Buttes. We spread out from Lamar Canyon West to Crystal Creek where Pauline and I heard a distant low howl. We went to Long Pullout and Rick found the Junctions at skyline south on Specimen just over the sheep cliff. They were just all bedded there happy snappy.
Things in report: {('Junctions', 'WLFP')}
Places in report: {'Lamar', 'Specimen', 'Slough'} 

paragraph #6
Mid-morning the Junctions all got up and headed east on Specimen. Calvin went to Slough to look but none of us were able to find them again. 
Things in report: {('Junctions', 'WLFP')}
Places in report: {'Specimen', 'Slough'} 

paragraph #7
Prospect ~While standing there watching the Junctio

### import nltk

sentence = "At eight o'clock on Thursday film morning word line test best beautiful Ram Aaron design"

tokens = nltk.word_tokenize(sentence)

tagged = nltk.pos_tag(tokens)

print(tagged)
length = len(tagged) - 1

a = list()

for item in tagged:
    if item[1][0] == 'N':
      a.append(item[0])
    
print(a)

In [ ]:
_POS_TAGGER = 'taggers/maxent_treebank_pos_tagger/english.pickle'
default_tagger = nltk.data.load(_POS_TAGGER)
model = {'Hi': 'VB'}
tagger = nltk.tag.UnigramTagger(model=model, backoff=default_tagger)

In [ ]:
#The one that works

import nltk.data, nltk.tag

from nltk.tag.perceptron import PerceptronTagger
class BackoffTagger:
    def __init__(self):
        self._taggers = [PerceptronTagger()]

entity_names = []

for entry in result['report']:
    tagged_sentences = []
    
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sent_list = sent_detector.tokenize(text.strip())   
    
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sent_list]
    

    tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]

    
    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)
    
    for word in chunked_sentences:
        print(word)
        entity_names.extend(extract_entity_names(word)) 
    break
    